# CNN_6channel

### Load scripts

In [1]:
%matplotlib inline
from script.util import *
from script.load_data_gray import *
from script.load_data_gray_filter import *

In [2]:
t0 = time.time()

run_name = get_run_name('SC_Iceberg_Classifier', 'Image_Proccess_Filter')
html_folder = get_html_folder(True)
input_folder = get_input_folder(True)
input_processed_folder = get_input_processed_folder(True)
log_folder = get_log_folder(True)
model_folder = get_model_folder(True)
output_folder = get_output_folder(True)

run_name: SC_Iceberg_Classifier-Image_Proccess_Filter-20180112_233110
D:\Kaggle\statoil-iceberg-classifier-challenge\html
D:\Kaggle\statoil-iceberg-classifier-challenge\input
D:\Kaggle\statoil-iceberg-classifier-challenge\input\processed
D:\Kaggle\statoil-iceberg-classifier-challenge\log
D:\Kaggle\statoil-iceberg-classifier-challenge\model
D:\Kaggle\statoil-iceberg-classifier-challenge\output


In [3]:
%%time
id_data, id_test = load_id()
y_data = load_y_data()

band1_data_gray, band2_data_gray, band1_test_gray, band2_test_gray = load_band_data_gray()
describe(band1_data_gray)
describe(band2_data_gray)
describe(band1_test_gray)
describe(band2_test_gray)

band_max_data = np.maximum(band1_data_gray, band2_data_gray)
band_max_test = np.maximum(band1_test_gray, band2_test_gray)
describe(band_max_data)
describe(band_max_test)

(1604,) 000d71b1 fffe76e4 12928
(8424,) 000c65d7 fff77564 67488
(1604,) 0 1 12928
(1604, 75, 75) 0.0 1.0 72180128
(1604, 75, 75) 0.0 1.0 72180128
(8424, 75, 75) 0.0 1.0 379080128
(8424, 75, 75) 0.0 1.0 379080128
(1604, 75, 75) 0.0 1.0 72180128
(1604, 75, 75) 0.0 1.0 72180128
(8424, 75, 75) 0.0 1.0 379080128
(8424, 75, 75) 0.0 1.0 379080128
(1604, 75, 75) 0.0210369138693 1.0 72180128
(8424, 75, 75) 0.0 1.0 379080128
Wall time: 3.59 s


In [4]:
%%time
band1_data_gray_sobel, band2_data_gray_sobel, band1_test_gray_sobel, band2_test_gray_sobel = load_band_data_gray_sobel()
describe(band1_data_gray_sobel)
describe(band2_data_gray_sobel)
describe(band1_test_gray_sobel)
describe(band2_test_gray_sobel)

band_max_data_sobel = np.maximum(band1_data_gray_sobel, band2_data_gray_sobel)
band_max_test_sobel = np.maximum(band1_test_gray_sobel, band2_test_gray_sobel)
describe(band_max_data_sobel)
describe(band_max_test_sobel)

(1604, 75, 75) 0.0 0.568412374825 72180128
(1604, 75, 75) 0.0 0.609926435782 72180128
(8424, 75, 75) 0.0 0.607657213723 379080128
(8424, 75, 75) 0.0 0.60653524554 379080128
(1604, 75, 75) 0.0 0.568412374825 72180128
(1604, 75, 75) 0.0 0.609926435782 72180128
(8424, 75, 75) 0.0 0.607657213723 379080128
(8424, 75, 75) 0.0 0.60653524554 379080128
(1604, 75, 75) 0.0 0.609926435782 72180128
(8424, 75, 75) 0.0 0.607657213723 379080128
Wall time: 2.02 s


In [5]:
inc_angle_data, inc_angle_test = load_inc_angle_data()
describe(inc_angle_data)
describe(inc_angle_test)

(1604,) 0.0 0.906748545018 12928
(8424,) 0.455580077133 1.0 67488
(1604,) 0.0 0.906748545018 12928
(8424,) 0.455580077133 1.0 67488


In [6]:
x_data = np.concatenate([
    band1_data_gray[:, :, :, np.newaxis],
    band2_data_gray[:, :, :, np.newaxis],
    band_max_data[:, :, :, np.newaxis]
#     band1_data_gray_sobel[:, :, :, np.newaxis],
#     band2_data_gray_sobel[:, :, :, np.newaxis],
#     band_max_data_sobel[:, :, :, np.newaxis]
], axis=-1)

x_test = np.concatenate([
    band1_test_gray[:, :, :, np.newaxis],
    band2_test_gray[:, :, :, np.newaxis],
    band_max_test[:, :, :, np.newaxis]
#     band1_test_gray_sobel[:, :, :, np.newaxis],
#     band2_test_gray_sobel[:, :, :, np.newaxis],
#     band_max_test_sobel[:, :, :, np.newaxis]
], axis=-1)

describe(x_data)
describe(x_test)

(1604, 75, 75, 3) 0.0 1.0 216540144
(8424, 75, 75, 3) 0.0 1.0 1137240144


### Build model

In [7]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Input
from keras.layers.merge import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard

Using TensorFlow backend.


In [8]:
def build_cnn(input_shape):
    bn_model = 0
    p_activation = "relu"
    input_1 = Input(shape=input_shape, name="bands")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation)(input_1)
    img_1 = BatchNormalization()(img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = BatchNormalization()(img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.4)(img_1)
    
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = BatchNormalization()(img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = BatchNormalization()(img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.4)(img_1)
    
    img_1 = Conv2D(128, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = BatchNormalization()(img_1)
    img_1 = Conv2D(128, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = BatchNormalization()(img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.4)(img_1)

    img_1 = GlobalAveragePooling2D() (img_1)
    
#     img_2 = Conv2D(128, kernel_size = (3,3), activation=p_activation)(input_1)
#     img_2 = BatchNormalization()(img_2)
#     img_2 = MaxPooling2D((2,2)) (img_2)
#     img_2 = Dropout(0.4)(img_2)
#     img_2 = GlobalMaxPooling2D() (img_2)
    
    angle = BatchNormalization()(input_2)
    img_concat = (Concatenate()([img_1, angle]))
    
#     img_concat = Concatenate([img_1, input_2])
    
    dense_ayer = Dense(1024, activation=p_activation)(img_concat)
    dense_ayer = BatchNormalization()(dense_ayer)
    dense_ayer = Dropout(0.4)(dense_ayer)
#     dense_ayer = Dense(1024, activation=p_activation)(dense_ayer)
#     dense_ayer = BatchNormalization()(dense_ayer)
#     dense_ayer = Dropout(0.4)(dense_ayer)
    output = Dense(1, activation="sigmoid")(dense_ayer)
    
    model = Model([input_1,input_2],  output)
#     optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#     model.compile(optimizer = Adam(lr=1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [9]:
def get_lr(x):
    lr = round(1e-4 * 0.995 ** x, 6)
    if lr < 1e-5:
        lr = 1e-5
    print(lr, end='  ')
    return lr

# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
annealer = LearningRateScheduler(get_lr)
callbacks = [annealer]

In [10]:
# datagen = ImageDataGenerator(
#     zoom_range = 0.2,
#     rotation_range=20,
#     horizontal_flip=True,
#     vertical_flip=True,
#     width_shift_range=0.2,
#     height_shift_range=0.2
# )

In [11]:
# %%time
# from sklearn.model_selection import train_test_split
# x_train, x_val, inc_angle_train, inc_angle_val, y_train, y_val = train_test_split(x_data_gray, inc_angle_data, y_data, test_size=0.1, random_state=31)
# describe(x_train)
# describe(x_val)
# describe(inc_angle_train)
# describe(inc_angle_val)
# describe(y_train)
# describe(y_val)

In [12]:
# hist = model.fit_generator(
#     datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True, seed=2019),
#     steps_per_epoch=steps_per_epoch,
#     epochs=200, #1 for ETA, 0 for silent
#     verbose=1,
#     max_queue_size=128,
#     callbacks=callbacks,
#     workers=32,
#     validation_data=(x_val, y_val)
# )

In [13]:
%%time
from sklearn.model_selection import train_test_split

def train_model(batch_size, epochs, x_data, inc_angle_data, y_data):
    model = build_cnn(x_data.shape[1:])
    model.compile(optimizer = Adam(lr=1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    x_train, x_val, inc_angle_train, inc_angle_val, y_train, y_val = train_test_split(x_data, inc_angle_data, y_data, test_size=0.1, random_state=2018)
    describe(x_train)
    describe(x_val)
    describe(inc_angle_train)
    describe(inc_angle_val)
    describe(y_train)
    describe(y_val)
#     steps_per_epoch = x_train.shape[0] / batch_size
    hist = model.fit(
        [x_train, inc_angle_train],
        y_train,
        batch_size = batch_size,
        verbose= 1,
        epochs = epochs,
        validation_data=([x_val, inc_angle_val], y_val),
        callbacks=callbacks
    )
    final_loss, final_acc = model.evaluate([x_val, inc_angle_val], y_val, verbose=1)
    print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))



train_model(8, 30, x_data, inc_angle_data, y_data)

(1443, 75, 75, 3) 0.0 1.0 194805144
(161, 75, 75, 3) 0.0 1.0 21735144
(1443,) 0.0 0.906748545018 11640
(161,) 0.0 0.906553131454 1384
(1443,) 0 1 11640
(161,) 0 1 1384
Train on 1443 samples, validate on 161 samples
0.0001  Epoch 1/30
1443/1443 [==============================] - 25s 18ms/step - loss: 0.7207 - acc: 0.6486 - val_loss: 0.9643 - val_acc: 0.4907
0.0001  Epoch 2/30
1443/1443 [==============================] - 22s 15ms/step - loss: 0.6355 - acc: 0.6881 - val_loss: 2.2056 - val_acc: 0.4907
9.9e-05  Epoch 3/30
1443/1443 [==============================] - 22s 15ms/step - loss: 0.6109 - acc: 0.7186 - val_loss: 1.4654 - val_acc: 0.4534
9.9e-05  Epoch 4/30
1443/1443 [==============================] - 22s 15ms/step - loss: 0.5771 - acc: 0.7270 - val_loss: 1.0004 - val_acc: 0.4845
9.8e-05  Epoch 5/30
1443/1443 [==============================] - 22s 15ms/step - loss: 0.5488 - acc: 0.7422 - val_loss: 1.3688 - val_acc: 0.5466
9.8e-05  Epoch 6/30
1443/1443 [==============================]

KeyboardInterrupt: 